<a href="https://colab.research.google.com/github/sseunghyuns/deeplearning/blob/master/nlp_sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Bidirectional, Embedding,LSTM

url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

with open('sarcasm.json') as f:
  data = json.load(f)

# DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

sentence =[]
label =[]

for d in data :
  sentence.append(d['headline'])
  label.append(d['is_sarcastic'])

In [15]:
# train / valid 나누기
training_size = 20000

train_sentence = sentence[:training_size]
train_label = label[:training_size]

valid_sentence = sentence[training_size:]
valid_label = label[training_size:]

In [16]:
# 토큰화 step-1 : Tokenizer 정의
vocab_size = 1000
oov_tok = "<OOV>"


tokenizer = Tokenizer(num_words = vocab_size,  # num_words =1000 개의 단어만 가져오겠다는 의미인지? 
                    oov_token = oov_tok)

In [17]:
# 토큰화 step-2 : 학습시킬 문장에 대해 토큰화 진행
tokenizer.fit_on_texts(train_sentence) # 단어 사전 만들기 

train_sequence = tokenizer.texts_to_sequences(train_sentence) # 단어 사전으로 문장 수치화. 사전에 없으면 1.
valid_sequence = tokenizer.texts_to_sequences(valid_sentence)

In [18]:
# 토큰화 step-3 : Sequence 길이 맞춰주기
max_length = 120 # 문장의 최대 길이
trunc_type='post' # 문장이 기준보다 길 때 앞을 자를지, 뒤를 자를지.
padding_type='post' #문장이 기준보다 짧을 때 앞을 채울지, 뒤를 채울지.

train_padded = pad_sequences(train_sequence, 
          maxlen = max_length, 
          truncating=trunc_type,
          padding=padding_type)

valid_padded = pad_sequences(valid_sequence,
                          padding=padding_type,
                          truncating=trunc_type, 
                          maxlen=max_length)

In [19]:
# 토큰화 step-4 : label 값을 array로 변환
train_label = np.array(train_label)
valid_label = np.array(valid_label)

In [20]:
# 모델 선언
model = Sequential([
                    Embedding(vocab_size,
                              embedding_dim,
                              input_length = max_length),
                    Bidirectional(LSTM(32)),
                    Dense(24, activation = 'relu'),
                    #Dense(16, activation = 'relu'),
                    Dense(1, activation = 'sigmoid')
])

In [21]:
# 모델 compile
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

In [22]:
# 모델 checkpoint
checkpath = 'ck.ckpt'
checkpoint = ModelCheckpoint(filepath = checkpath,
                              monitor='val_loss',
                              save_best_only = True,
                              save_weights_only = True,
                              verbose= 1)

In [23]:
# 모델 학습
model.fit(train_padded, train_label,
          validation_data = (valid_padded, valid_label),
          callbacks = [checkpoint],
          epochs = 20)

# 모델 load
model.load_weights(checkpath)

Epoch 1/20
625/625 [==============================] - ETA: 0s - loss: 0.4526 - acc: 0.7728
Epoch 00001: val_loss improved from inf to 0.39376, saving model to ck.ckpt
625/625 [==============================] - 37s 59ms/step - loss: 0.4526 - acc: 0.7728 - val_loss: 0.3938 - val_acc: 0.8207
Epoch 2/20
625/625 [==============================] - ETA: 0s - loss: 0.3574 - acc: 0.8353
Epoch 00002: val_loss improved from 0.39376 to 0.37999, saving model to ck.ckpt
625/625 [==============================] - 37s 59ms/step - loss: 0.3574 - acc: 0.8353 - val_loss: 0.3800 - val_acc: 0.8232
Epoch 3/20
624/625 [============================>.] - ETA: 0s - loss: 0.3383 - acc: 0.8473
Epoch 00003: val_loss improved from 0.37999 to 0.37528, saving model to ck.ckpt
625/625 [==============================] - 37s 59ms/step - loss: 0.3383 - acc: 0.8473 - val_loss: 0.3753 - val_acc: 0.8290
Epoch 4/20
625/625 [==============================] - ETA: 0s - loss: 0.3205 - acc: 0.8551
Epoch 00004: val_loss improved 